# Task2 同名消歧

最新成绩，baseline1直接获得
- score 0.24865
- nickname awyd234
- rank 48

In [90]:
import os
import json

## 数据集读取

In [91]:
train_author_file_path = './files/name_disambiguation/train_author.json'
train_pub_file_path = './files/name_disambiguation/train_pub.json'

with open(train_author_file_path) as f:
    train_author_dict = json.load(f)
    
with open(train_pub_file_path) as f:
    train_pub_dict = json.load(f)

In [92]:
train_author_dict['li_guo']['EShnTfSe']

['P9a1gcvg']

In [93]:
train_pub_dict['P9a1gcvg']

{'authors': [{'name': 'Fenghe Qiu',
   'org': 'Institute of Pharmacology and Toxicology'},
  {'name': 'Li Liu', 'org': 'Institute of Pharmacology and Toxicology'},
  {'name': 'Li Guo', 'org': 'Institute of Pharmacology and Toxicology'}],
 'title': 'Rapid determination of central nervous drugs in plasma by solid-phase extraction and GC-FID and GC-MS',
 'abstract': 'Objective: To establish a simultaneous determination method of central nervous drugs including barbitals, benzodiazepines, phenothiozines and tricyclic antidepressants in human plasma. Methods: Drugs in plasma were extracted and purified by using X-5 resin solid phase extraction columns, followed by identification and quantitation using capillary GC-FID and GC-MS. Results: More than 20 drugs were simultaneously extracted from human plasma, and effectively separated in GC and TIC spectra. The correlation coefficient of standard curves was larger than 0.99, and relative standard differences (RSD) were less than 10% for most of 

In [114]:
example_upload_file_path = './files/name_disambiguation/sna_valid_example_evaluation_scratch.json'
with open(example_upload_file_path, 'r') as f:
    example_upload_dict = json.load(f)
json.dumps(example_upload_dict);

In [95]:
sns_valid_pub_file_path = './files/name_disambiguation/sna_valid_pub.json'
with open(sns_valid_pub_file_path, 'r') as f:
    sns_valid_pub_dict = json.load(f)

In [96]:
sns_valid_author_file_path = './files/name_disambiguation/sna_valid_author_raw.json'
with open(sns_valid_author_file_path, 'r') as f:
    sns_valid_author_dict = json.load(f)

In [97]:
def save_json(obj, path, indent=2, ensure_ascii=False, override=True, *args, **kwargs):
    """Save object as json format"""
    if not override:
        if os.path.exists(path):
            return
    with open(path, "w") as f:
        json.dump(obj, f, indent=indent, ensure_ascii=ensure_ascii, *args, **kwargs)

In [113]:
print(sns_valid_author_dict.keys())
print(sns_valid_author_dict['heng_li'][:10])

dict_keys(['heng_li', 'fang_chen', 'lin_zhou', 'akio_yamamoto', 'y_luo', 'chun_li', 'shiyi_chen', 'zhigang_chen', 'chun_wang', 'j_yu', 'y_guo', 'g_li', 'jing_huang', 'atsushi_takeda', 'fei_gao', 'rajendra_prasad', 'qi_li', 'bin_ren', 'yong_cao', 'jing_zhou', 'hong_jiang', 'liang_zhou', 'ming_xu', 'jie_sun', 'hao_zhang', 'jue_wang', 'y_shi', 'jian_huang', 'ping_zhang', 'feng_zhou', 'dong_zhang', 'h_yu', 'jian_wu', 'li_huang', 'shiyong_liu', 'ke_xu', 'min_yang', 'lei_wu', 'jie_zhou', 'zhang_lei', 'h_y_wang', 'qing_li', 'd_wang', 'hongsheng_chen', 'kun_li', 'hong_guo', 'gang_chen', 'yuanyuan_zhang', 'lin_ma', 'yao_zhang'])
['zszavvJh', 'a2Tl88Xu', 'X3Y0AYBw', '0kUbjIMW', '4bKZMNxE', 'vbQeLjWI', 'ZfSbG5ME', 'ierOTBz1', '6cXZA7zu', 'cXBXYa63']


## Baseline1 

完全不做处理，直接认为同名即同一人

- score 0.24865
- nickname awyd234
- rank 23
- datetime 2020-05-25 05:24

In [99]:
result_dict1 = {}
for key, value in sns_valid_author_dict.items():
    result_dict1[key] = [value]

In [100]:
upload_dir = './files/name_disambiguation/upload'
if not os.path.exists(upload_dir):
    os.mkdir(upload_dir)

In [101]:
save_json(result_dict1, os.path.join(upload_dir, 'baseline1.json'))

## Baseline2 尝试中

按照同样的组织名进行归类，需要检测作者和pub里的作者是不是同一人，以及对没有结果的进行处理；尝试了很长时间对姓名进行处理归类，成绩还不如直接不分类提交，有点自闭，看来还是得上聚类

- score 0.11718
- nickname awyd234
- rank null
- datetime 2020-05-28 13:37

In [102]:
all_valid_names_list = list(sns_valid_author_dict.keys())
print(all_valid_names_list)

['heng_li', 'fang_chen', 'lin_zhou', 'akio_yamamoto', 'y_luo', 'chun_li', 'shiyi_chen', 'zhigang_chen', 'chun_wang', 'j_yu', 'y_guo', 'g_li', 'jing_huang', 'atsushi_takeda', 'fei_gao', 'rajendra_prasad', 'qi_li', 'bin_ren', 'yong_cao', 'jing_zhou', 'hong_jiang', 'liang_zhou', 'ming_xu', 'jie_sun', 'hao_zhang', 'jue_wang', 'y_shi', 'jian_huang', 'ping_zhang', 'feng_zhou', 'dong_zhang', 'h_yu', 'jian_wu', 'li_huang', 'shiyong_liu', 'ke_xu', 'min_yang', 'lei_wu', 'jie_zhou', 'zhang_lei', 'h_y_wang', 'qing_li', 'd_wang', 'hongsheng_chen', 'kun_li', 'hong_guo', 'gang_chen', 'yuanyuan_zhang', 'lin_ma', 'yao_zhang']


In [103]:
# 检查验证集中author是否有同名
all_names_set = set()
for each_name in all_valid_names_list:
    name_split = each_name.split('_')
    if len(name_split) > 2:
        print("name split greater than 2", each_name)
        all_names_set.add(each_name)
        continue
    first, last = name_split
    first = first.lower()
    last = first.lower()
    second_name = '{last}_{first}'.format(**locals())
    if each_name not in all_names_set:
        all_names_set.add(each_name)
    else:
        print('first name exists', each_name)
    if second_name not in all_names_set:
        all_names_set.add(second_name)
    else:
        print('second name exists', second_name)

second name exists chun_chun
second name exists y_y
second name exists jing_jing
second name exists y_y
second name exists jian_jian
second name exists jie_jie
name split greater than 2 h_y_wang
second name exists hong_hong
second name exists lin_lin


In [104]:
sns_valid_pub_dict['iBgl2DTz']

{'id': 'iBgl2DTz',
 'title': 'A generalized extended state observer for supercapacitor state of charge estimation under disturbances.',
 'abstract': 'In this paper, a generalized extended state observer (GESO) is proposed to accurately observe supercapacitoru0027s state of charge (SOC) by counteracting the modeling deviation and disturbances. With the consideration of dynamic modeling uncertainty and disturbances, a nonlinear mathematical model is established based on the three branch equivalent circuit model for supercapacitor (SC). A GESO is designed to observe SCu0027s internal capacitance voltage and modeling error with the known information from the established mathematical model. With the estimated modeling error information, a compensation control law is designed to eliminate deviation, which results in an accurate internal capacitance voltage observation. An accurate SOC for SC is obtained with the estimated parameters. Experimental results from a synchronous buck circuit based

In [105]:
def check_if_same_person(name_a, name_b):
    name_b = name_b.replace('.', ' ').replace('\xa0', ' ').replace('-', '')
    name_a_split = name_a.split('_')
    name_b_split = [_ for _ in name_b.split(' ') if _.strip() != '']
    name_b_lower_list = [_.lower() for _ in name_b_split]
    if name_a == name_b:
        return True
    if '_'.join(name_b_lower_list) == name_a or '_'.join(list(reversed(name_b_lower_list))) == name_a:
        return True
    if len(name_a_split) != 0 and len(name_b_lower_list) != 0:
        if name_a_split[-1] == name_b_lower_list[-1]:
            return True
#     print(name_a, '_'.join(name_b_lower_list), '_'.join(list(reversed(name_b_lower_list))))
    return False

In [106]:
import random

result_dict2 = {}
this_count = {
    '0': 0,
    '1': 0
}
for key, value in sns_valid_author_dict.items():
    org_result_dict = {}
    for each_pub_id in value:
        pub_info_dict = sns_valid_pub_dict[each_pub_id]
        authors_list = pub_info_dict['authors']
        flag = False
        for each_author_info in authors_list:
            if check_if_same_person(key, each_author_info['name']):
                flag = True
                org_name = each_author_info.get('org')
                if org_result_dict.get(org_name) is None:
                    org_result_dict[org_name] = []
                org_result_dict[org_name].append(each_pub_id)
                break
        if not flag:
            this_count['0'] += 1
            print(each_pub_id, key, [_['name'] for _ in authors_list])
            if len(org_result_dict) == 0:
                org_result_dict[org_name] = [each_pub_id]
            else:
                org_result_dict[random.choice(list(org_result_dict.keys()))].append(each_pub_id)
    result_dict2[key] = list(org_result_dict.values())
print(this_count)

v8MQ189W heng_li ['Jing YH', 'Yang XY', 'Zhao CR', 'Li H', 'Guo MF', 'Wang Q', 'Wang Q']
47dPthKZ heng_li ['Gao Lei', 'l H']
EibaQ9x1 heng_li ['LI Hong-bin', 'ZHOU You-qiang', 'LI H']
PnYTYzt6 heng_li ['ZHANG Bofeng', 'ZHANG Yingze', 'QI Xiangbei', 'LI H']
XpeXTCJI heng_li ['Gao Lei', 'l H']
hPphSy2I zhigang_chen ['hua huang', 'dc feng', 'gy luo', 'zg cheng', 'lr lei', 'youdi liao', 'yj zhang', 'am yu', 'j tan', 'zb yang', 'x jin', 'fb meng']
vQgXWHC1 yong_cao ['Kafafy R.', 'Cao Y.']
vS23uZbV yong_cao ['', '', '', '']
g0bizfjw yong_cao ['Yu D.', 'Liu H.', 'Cao Y.', 'Fu H.']
MJ8zE8I7 hong_jiang ['Qian XH', 'Tang ZY', 'Ye XL', 'Bi LY', 'Wu T', 'Jiang H', 'Shi Q']
xggdEb32 hong_jiang ['', '']
hIMn9QHK hong_jiang ['', '']
0OVUOlLX hao_zhang ['', '', '', '', '', '']
Co5aMeOa hao_zhang ['', '']
uIJ0iovn hao_zhang ['Changqing Zou', 'Haoran Mo', 'Ruofei Du', 'Xing Wu', 'Chengying Gao', 'Hongbo Fu']
2ZZ5oS74 hao_zhang ['', '', '', '']
mEIocqZe feng_zhou ['ZHOU F', 'HU Yang', 'SUN Zhi-gang']
BIQ

In [107]:
save_json(result_dict2, os.path.join(upload_dir, 'baseline2.json'))

In [108]:
count = 0
for k, v in result_dict2.items():
    count += sum([len(_) for _ in v])
print(count)

45898


In [109]:
sns_valid_pub_dict['zOyScSRD']

{'id': 'zOyScSRD',
 'title': 'Recurrent intracranial hemangiopericytoma with multiple metastases.',
 'abstract': 'null',
 'keywords': ['neoplasm metastasis', 'hemangiopericytoma'],
 'authors': [{'name': 'cao', 'org': ''},
  {'name': 'yong', 'org': ''},
  {'name': 'zhang', 'org': ''},
  {'name': 'maozhi', 'org': ''},
  {'name': 'wang', 'org': ''},
  {'name': 'junmei', 'org': ''},
  {'name': 'wei', 'org': ''},
  {'name': 'li', 'org': ''},
  {'name': 'guang', 'org': ''},
  {'name': 'zhao', 'org': ''},
  {'name': 'jizong', 'org': ''}],
 'venue': 'Chinese medical journal',
 'year': 2006}